## Day 6

https://adventofcode.com/2021/day/6

In [1]:
timerlist0 = [3,4,3,1,2]

with open("data/input06.txt") as f:
    timerlist = [ int(t) for t in f.readlines()[0].split(",") ]

### Fast solution of both Part 1 and Part 2

There is no real need of `LanternFish` class as in my first attempt (see below), fish timer counts can be simply handled in an array! 

In [2]:
import time

def solve6Simple(timerlist,days):
    timers = [0]*9
    for f in timerlist:
        timers[f] += 1
    for d in range(days):
        newtimers = timers[1:]+[timers[0]] # shift all counts to the left (e.g. decrease timers) + add offspring
        newtimers[6] += timers[0] # reset timers for fish at end of cycle
        timers = newtimers # rinse and repeat
    return(sum(timers))

start_time = time.time()
print("Test 1:",solve6Simple(timerlist0,80))
print("Part 1:",solve6Simple(timerlist,80))
print("Test 2:",solve6Simple(timerlist0,256))
print("Part 2:",solve6Simple(timerlist,256))
print("Part 2 time: %s s" % (time.time() - start_time))

Test 1: 5934
Part 1: 394994
Test 2: 26984457539
Part 2: 1765974267455
Part 2 time: 0.0018301010131835938 s


### Solution with linear algebra

In [16]:
import numpy as np

# one matrix multiplication shift vector elements to the left, 
# plus add values at 0 in position 6. e.g. e day fish population evolution

dayiter = np.array([[0,1,0,0,0,0,0,0,0],
                    [0,0,1,0,0,0,0,0,0],
                    [0,0,0,1,0,0,0,0,0],
                    [0,0,0,0,1,0,0,0,0],
                    [0,0,0,0,0,1,0,0,0],
                    [0,0,0,0,0,0,1,0,0],
                    [1,0,0,0,0,0,0,1,0], # also add values at 0 to counter 6
                    [0,0,0,0,0,0,0,0,1],
                    [1,0,0,0,0,0,0,0,0]])

def solve6Matrix(timerlist,days):
    timers = np.array([0]*9)
    for f in timerlist:
        timers[f] += 1
    # instead of n_days matrix multiplications, use matrix power
    return sum(np.matmul(np.linalg.matrix_power(dayiter,days),timers))

print("Test 1:",solve6Matrix(timerlist0,80))
print("Part 1:",solve6Matrix(timerlist,80))
print("Test 2:",solve6Matrix(timerlist0,256))
start_time = time.time()
print("Part 2:",solve6Matrix(timerlist,256))
print("Part 2 time: %s s" % (time.time() - start_time))

Test 1: 5934
Part 1: 394994
Test 2: 26984457539
Part 2: 1765974267455
Part 2 time: 0.0003409385681152344 s


### Part 1 initial solution

First attempt using a class, iterating on all population (class is overkill, procedure not very efficient!)

In [4]:
class LanternFish:
    def __init__(self,timer=6):
        self.timer = timer
    def timePass(self):
        self.timer -= 1
        if self.timer <0:
            self.timer = 6
            return LanternFish(8)
        else:
            return None

In [5]:
import inflect
p = inflect.engine()

def solve6Slow(timerlist,days=18,verbose=True):
    fishes = [ LanternFish(t) for t in timerlist ]
    for d in range(days):
        for f in list(fishes):
            spawn = f.timePass()
            if spawn:
                fishes.append(spawn)
        if verbose:
            print("After {:3d} {:4s} : ".format(d+1,p.plural("day") if d+1>1 else "day"),end="")
            for f in fishes:
                print(f.timer,end=" ")
            print()
    return len(fishes)

In [6]:
solve6Slow(timerlist0,days=18,verbose=True)

After   1 day  : 2 3 2 0 1 
After   2 days : 1 2 1 6 0 8 
After   3 days : 0 1 0 5 6 7 8 
After   4 days : 6 0 6 4 5 6 7 8 8 
After   5 days : 5 6 5 3 4 5 6 7 7 8 
After   6 days : 4 5 4 2 3 4 5 6 6 7 
After   7 days : 3 4 3 1 2 3 4 5 5 6 
After   8 days : 2 3 2 0 1 2 3 4 4 5 
After   9 days : 1 2 1 6 0 1 2 3 3 4 8 
After  10 days : 0 1 0 5 6 0 1 2 2 3 7 8 
After  11 days : 6 0 6 4 5 6 0 1 1 2 6 7 8 8 8 
After  12 days : 5 6 5 3 4 5 6 0 0 1 5 6 7 7 7 8 8 
After  13 days : 4 5 4 2 3 4 5 6 6 0 4 5 6 6 6 7 7 8 8 
After  14 days : 3 4 3 1 2 3 4 5 5 6 3 4 5 5 5 6 6 7 7 8 
After  15 days : 2 3 2 0 1 2 3 4 4 5 2 3 4 4 4 5 5 6 6 7 
After  16 days : 1 2 1 6 0 1 2 3 3 4 1 2 3 3 3 4 4 5 5 6 8 
After  17 days : 0 1 0 5 6 0 1 2 2 3 0 1 2 2 2 3 3 4 4 5 7 8 
After  18 days : 6 0 6 4 5 6 0 1 1 2 6 0 1 1 1 2 2 3 3 4 6 7 8 8 8 8 


26

In [7]:
solve6Slow(timerlist0,days=80,verbose=False)

5934

In [8]:
solve6Slow(timerlist,days=80,verbose=False)

394994

### Part 2 initial solution

Storing fish population information in a dictionary to avoid exponential growth...

In [9]:
from collections import defaultdict

def solve6Fast(timerlist,days=80):
    fishPop = defaultdict(lambda: 0)
    for t in timerlist:
        fishPop[t] += 1
    for d in range(days):
        fishPopNew = defaultdict(lambda: 0)
        for f in fishPop.keys():
            thisFish = LanternFish(f)
            spawn = thisFish.timePass()
            fishPopNew[thisFish.timer] += fishPop[f]
            if spawn:
                fishPopNew[spawn.timer] += fishPop[f]
        fishPop = fishPopNew
    return sum([ fishPop[f] for f in fishPop.keys() ])

In [10]:
print("Test 2:",solve6Fast(timerlist0,256))
start_time = time.time()
print("Part 2:",solve6Fast(timerlist,256))
print("Part 2 time: %s s" % (time.time() - start_time))

Test 2: 26984457539
Part 2: 1765974267455
Part 2 time: 0.002980947494506836 s
